## Importacao de Bibliotecas

Carrega todas as bibliotecas necessarias para o pipeline de imputacao. O modulo `enable_iterative_imputer` deve ser importado antes do `IterativeImputer` pois este ainda e considerado experimental no scikit-learn. O `RandomForestRegressor` sera utilizado como estimador base do imputador iterativo. As bibliotecas matplotlib e seaborn sao utilizadas para visualizacao dos resultados da avaliacao.

In [1]:
import os
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import pickle
import time
import warnings
from glob import glob
import boto3
from botocore.exceptions import ClientError

warnings.filterwarnings('ignore')

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import matplotlib.pyplot as plt
import seaborn as sns
import mlflow
import mlflow.sklearn

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

# Variáveis de ambiente já estão configuradas no docker-compose.yml
S3_BUCKET_NAME = os.getenv('S3_BUCKET_NAME') or os.getenv('AWS_BUCKET_NAME')
AWS_ACCESS_KEY_ID = os.getenv('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = os.getenv('AWS_SECRET_ACCESS_KEY')
AWS_REGION = os.getenv('AWS_REGION', 'us-east-2')
S3_PREFIX = os.getenv('S3_PREFIX', 'dados_imputados')

print(f"Configuração AWS S3:")
print(f"  Bucket: {S3_BUCKET_NAME}")
print(f"  Region: {AWS_REGION}")
print(f"  Prefix: {S3_PREFIX}")

Configuração AWS S3:
  Bucket: avd-5b
  Region: us-east-2
  Prefix: dados_imputados


In [2]:
EVALUATION_RATIO = 0.10
RANDOM_STATE = 42

print(f"Configuracao do Pipeline de Imputacao Automatizado")
print(f"  Ratio de avaliacao: {EVALUATION_RATIO * 100}%")
print(f"  Random state: {RANDOM_STATE}")

Configuracao do Pipeline de Imputacao Automatizado
  Ratio de avaliacao: 10.0%
  Random state: 42


In [3]:
MODELOS_CONFIG = [
    {'nome': 'Modelo_Leve', 'n_estimators': 5, 'max_depth': 2, 'max_iter': 2},
    {'nome': 'Modelo_Medio', 'n_estimators': 10, 'max_depth': 3, 'max_iter': 3},
    {'nome': 'Modelo_Robusto', 'n_estimators': 15, 'max_depth': 4, 'max_iter': 4}
]

print(f"Configuracoes de modelos definidas: {len(MODELOS_CONFIG)} modelos")
for config in MODELOS_CONFIG:
    print(f"  {config['nome']}: n_estimators={config['n_estimators']}, max_depth={config['max_depth']}, max_iter={config['max_iter']}")

Configuracoes de modelos definidas: 3 modelos
  Modelo_Leve: n_estimators=5, max_depth=2, max_iter=2
  Modelo_Medio: n_estimators=10, max_depth=3, max_iter=3
  Modelo_Robusto: n_estimators=15, max_depth=4, max_iter=4


## Descoberta Automatica de Estacoes

Detecta automaticamente todas as estacoes disponiveis com base nos arquivos PKL gerados pelo notebook de tratamento. O sistema busca por arquivos no formato `dados_tratados_*.pkl` e extrai os nomes das estacoes para processamento em lote.

In [4]:
dados_files = glob('dados_tratados_*.pkl')
estacoes_disponiveis = []

for file in dados_files:
    estacao_name = file.replace('dados_tratados_', '').replace('.pkl', '')
    estacoes_disponiveis.append(estacao_name)

estacoes_disponiveis = sorted(estacoes_disponiveis)

print(f"\nEstacoes detectadas: {len(estacoes_disponiveis)}")
for i, estacao in enumerate(estacoes_disponiveis, 1):
    print(f"  {i}. {estacao.replace('_', ' ')}")


Estacoes detectadas: 12
  1. ARCO VERDE
  2. CABROBO
  3. CARUARU
  4. FLORESTA
  5. GARANHUNS
  6. IBIMIRIM
  7. OURICURI
  8. PALMARES
  9. PETROLINA
  10. SALGUEIRO
  11. SERRA TALHADA
  12. SURUBIM


## Inicializacao do MLflow

Inicializa o servico MLflow para rastreamento de experimentos, parametros e metricas de cada estacao processada.

In [5]:
# mlflow_service.initialize()

# print(f"MLflow inicializado")
# print(f"  Tracking URI: {mlflow_service.tracking_uri}")
# print(f"  Experimento: Imputacao por Estacao")

mlflow.set_tracking_uri("http://mlflow:5000")

EXPERIMENT_NAME = "Imputacao_por_Estacao"

try:
    exp_id = mlflow.create_experiment(EXPERIMENT_NAME)
except:
    exp_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

mlflow.set_experiment(experiment_id=exp_id)

print(f"MLflow inicializado")
print(f"  Tracking URI: http://mlflow:5000")
print(f"  Experimento: {EXPERIMENT_NAME}")

MLflow inicializado
  Tracking URI: http://mlflow:5000
  Experimento: Imputacao_por_Estacao


## Funcao de Carregamento de Dados da Estacao

Define a funcao que carrega o dataset e metadados de uma estacao especifica a partir dos arquivos PKL gerados no notebook de tratamento.

In [6]:
def carregar_dados_estacao(estacao_filename):
    """
    Carrega dados e metadados de uma estacao.
    
    Args:
        estacao_filename: Nome do arquivo da estacao (ex: 'PETROLINA')
        
    Returns:
        tuple: (df, metadata)
    """
    pkl_filename = f'dados_tratados_{estacao_filename}.pkl'
    metadata_filename = f'metadata_tratamento_{estacao_filename}.pkl'
    
    df = pd.read_pickle(pkl_filename)
    with open(metadata_filename, 'rb') as f:
        metadata = pickle.load(f)
    
    return df, metadata

print("Funcao carregar_dados_estacao() definida")

Funcao carregar_dados_estacao() definida


## Funcao de Mascaramento para Avaliacao

Define a funcao que mascara uma fracao dos valores conhecidos para avaliar a qualidade da imputacao. Esta tecnica permite comparar valores imputados com valores reais, calculando metricas de erro objetivas.

In [7]:
def mascarar_dados(df, target_cols, evaluation_ratio, random_state):
    """
    Mascara valores para avaliacao da imputacao.
    
    Args:
        df: DataFrame com os dados
        target_cols: Lista de colunas alvo
        evaluation_ratio: Percentual de dados para mascarar
        random_state: Seed para reproducibilidade
        
    Returns:
        tuple: (df_masked, masked_data)
    """
    np.random.seed(random_state)
    df_masked = df.copy()
    masked_data = {}
    
    for col in target_cols:
        non_null_indices = df[col].dropna().index
        n_to_mask = int(len(non_null_indices) * evaluation_ratio)
        mask_indices = np.random.choice(non_null_indices, size=n_to_mask, replace=False)
        
        masked_data[col] = {
            'indices': mask_indices,
            'true_values': df_masked.loc[mask_indices, col].copy()
        }
        df_masked.loc[mask_indices, col] = np.nan
    
    return df_masked, masked_data

print("Funcao mascarar_dados() definida")

Funcao mascarar_dados() definida


## Funcao de Configuracao do Imputador

Define a funcao que cria e configura o IterativeImputer com RandomForestRegressor como estimador base. Os parametros foram escolhidos para balancear qualidade de imputacao e tempo de processamento.

In [8]:
def criar_imputador(config, random_state):
    """
    Cria e configura o IterativeImputer com base na configuracao fornecida.
    
    Args:
        config: Dicionario com parametros do modelo
        random_state: Seed para reproducibilidade
        
    Returns:
        IterativeImputer configurado
    """
    rf_estimator = RandomForestRegressor(
        n_estimators=config['n_estimators'],
        max_depth=config['max_depth'],
        min_samples_split=10,
        min_samples_leaf=5,
        random_state=random_state,
        n_jobs=-1
    )
    
    imputer = IterativeImputer(
        estimator=rf_estimator,
        max_iter=config['max_iter'],
        random_state=random_state,
        verbose=0
    )
    
    return imputer

print("Funcao criar_imputador() definida")

Funcao criar_imputador() definida


## Funcao de Execucao da Imputacao

Define a funcao que executa o processo de imputacao nos dados mascarados, preservando as colunas identificadoras (id, data, hora).

In [9]:
def executar_imputacao(imputer, df_masked, target_cols, feature_cols):
    """
    Executa a imputacao nos dados mascarados.
    
    Args:
        imputer: IterativeImputer configurado
        df_masked: DataFrame com valores mascarados
        target_cols: Lista de colunas alvo
        feature_cols: Lista de features
        
    Returns:
        DataFrame com valores imputados
    """
    columns_for_imputation = target_cols + feature_cols
    X_masked = df_masked[columns_for_imputation].copy()
    
    X_imputed = imputer.fit_transform(X_masked)
    
    df_imputed = pd.DataFrame(X_imputed, columns=columns_for_imputation, index=df_masked.index)
    df_imputed['id'] = df_masked['id']
    df_imputed['data'] = df_masked['data']
    df_imputed['hora'] = df_masked['hora']
    
    return df_imputed

print("Funcao executar_imputacao() definida")

Funcao executar_imputacao() definida


## Funcao de Calculo de Metricas

Define a funcao que calcula as metricas de qualidade da imputacao comparando valores imputados com valores reais mascarados. As metricas incluem RMSE, MAE e R-quadrado.

In [10]:
def calcular_metricas(df_imputed, masked_data, target_cols):
    """
    Calcula metricas de avaliacao da imputacao.
    
    Args:
        df_imputed: DataFrame com valores imputados
        masked_data: Dicionario com valores mascarados
        target_cols: Lista de colunas alvo
        
    Returns:
        dict: Metricas de avaliacao por coluna
    """
    evaluation_results = {}
    
    for col in target_cols:
        indices = masked_data[col]['indices']
        true_values = masked_data[col]['true_values']
        imputed_values = df_imputed.loc[indices, col]
        
        rmse = np.sqrt(mean_squared_error(true_values, imputed_values))
        mae = mean_absolute_error(true_values, imputed_values)
        r2 = r2_score(true_values, imputed_values)
        
        evaluation_results[col] = {'RMSE': rmse, 'MAE': mae, 'R2': r2}
    
    return evaluation_results

print("Funcao calcular_metricas() definida")

Funcao calcular_metricas() definida


## Funcao de Geracao de Visualizacao

Define a funcao que gera graficos de dispersao comparando valores reais versus valores imputados para cada variavel alvo. A linha diagonal representa a predicao perfeita.

In [11]:
def gerar_visualizacao(df_imputed, masked_data, target_cols, evaluation_results, estacao_nome, estacao_filename):
    """
    Gera graficos de avaliacao da imputacao.
    
    Args:
        df_imputed: DataFrame com valores imputados
        masked_data: Dicionario com valores mascarados
        target_cols: Lista de colunas alvo
        evaluation_results: Metricas de avaliacao
        estacao_nome: Nome da estacao
        estacao_filename: Nome do arquivo da estacao
        
    Returns:
        str: Caminho do arquivo de plot salvo
    """
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    fig.suptitle(f'Avaliacao da Imputacao - {estacao_nome}', fontsize=14, fontweight='bold')
    
    for idx, col in enumerate(target_cols):
        ax = axes[idx]
        indices = masked_data[col]['indices']
        true_values = masked_data[col]['true_values']
        imputed_values = df_imputed.loc[indices, col]
        
        ax.scatter(true_values, imputed_values, alpha=0.5, s=20)
        min_val = min(true_values.min(), imputed_values.min())
        max_val = max(true_values.max(), imputed_values.max())
        ax.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, alpha=0.7)
        
        ax.set_xlabel('Valor Real', fontsize=11, fontweight='bold')
        ax.set_ylabel('Valor Imputado', fontsize=11, fontweight='bold')
        ax.set_title(
            f"{col.upper()}\nRMSE: {evaluation_results[col]['RMSE']:.3f} | R2: {evaluation_results[col]['R2']:.3f}",
            fontsize=12, fontweight='bold'
        )
        ax.grid(True, alpha=0.3)
        ax.set_aspect('equal', adjustable='box')
    
    plt.tight_layout()
    output_plot = f'avaliacao_imputacao_{estacao_filename}.png'
    plt.savefig(output_plot, dpi=150, bbox_inches='tight')
    plt.close()
    
    return output_plot

print("Funcao gerar_visualizacao() definida")

Funcao gerar_visualizacao() definida


## Funcao de Exportacao de Resultados

Define a funcao que exporta os resultados da imputacao para arquivos locais, incluindo o dataset completo, dados para update no banco e metricas de avaliacao.

In [12]:
def exportar_resultados(df_final, evaluation_results, estacao_filename):
    """
    Exporta resultados da imputacao para arquivos.
    
    Args:
        df_final: DataFrame com dados imputados finais
        evaluation_results: Metricas de avaliacao
        estacao_filename: Nome do arquivo da estacao
        
    Returns:
        dict: Caminhos dos arquivos exportados
    """
    df_for_update = pd.DataFrame({
        'id': df_final['id'],
        'data': df_final['data'],
        'hora': df_final['hora'],
        'temperatura': df_final['temperatura'],
        'umidade': df_final['umidade'],
        'velocidade_vento': df_final['velocidade_vento']
    })
    
    output_pkl = f'dados_imputados_{estacao_filename}.pkl'
    output_csv = f'dados_para_update_neon_{estacao_filename}.csv'
    output_metrics = f'metricas_imputacao_{estacao_filename}.pkl'
    
    df_final.to_pickle(output_pkl)
    df_for_update.to_csv(output_csv, index=False)
    with open(output_metrics, 'wb') as f:
        pickle.dump(evaluation_results, f)
    
    return {
        'pkl': output_pkl,
        'csv': output_csv,
        'metrics': output_metrics
    }

print("Funcao exportar_resultados() definida")

Funcao exportar_resultados() definida


## Funcao Principal de Processamento

Define a funcao principal que orquestra todo o pipeline de imputacao para uma estacao: carregamento, mascaramento, imputacao, avaliacao, visualizacao, exportacao e registro no MLflow.

In [13]:
def processar_estacao(estacao_filename, evaluation_ratio, random_state):
    """
    Processa imputacao completa para uma estacao com tres modelos diferentes.
    
    Args:
        estacao_filename: Nome do arquivo da estacao
        evaluation_ratio: Percentual de dados para mascaramento
        random_state: Seed para reproducibilidade
        
    Returns:
        dict: Metricas de avaliacao por modelo
    """
    print("\n" + "=" * 70)
    print(f"PROCESSANDO ESTACAO: {estacao_filename.replace('_', ' ')}")
    print("=" * 70)
    
    start_time = time.time()
    
    df, metadata = carregar_dados_estacao(estacao_filename)
    target_cols = metadata['target_cols']
    feature_cols = metadata['feature_cols']
    estacao_nome = metadata['estacao']
    
    print(f"\n1. Dataset carregado: {df.shape}")
    print(f"   Registros: {len(df):,}")
    print(f"   Periodo: {metadata['date_range'][0]} ate {metadata['date_range'][1]}")
    
    df_original = df.copy()
    df_masked, masked_data = mascarar_dados(df, target_cols, evaluation_ratio, random_state)
    
    print(f"\n2. Mascaramento para avaliacao ({evaluation_ratio*100}%):")
    for col in target_cols:
        n_masked = len(masked_data[col]['indices'])
        print(f"   {col}: {n_masked:,} valores mascarados")
    
    # Processar os tres modelos
    resultados_modelos = {}
    
    for idx, config in enumerate(MODELOS_CONFIG, 1):
        modelo_nome = config['nome']
        print(f"\n{'='*70}")
        print(f"MODELO {idx}/3: {modelo_nome}")
        print(f"{'='*70}")
        
        modelo_start = time.time()
        
        imputer = criar_imputador(config, random_state)
        
        print(f"\n3. Executando imputacao com {modelo_nome}...")
        df_imputed = executar_imputacao(imputer, df_masked, target_cols, feature_cols)
        
        print(f"\n4. Calculando metricas de avaliacao:")
        evaluation_results = calcular_metricas(df_imputed, masked_data, target_cols)
        for col in target_cols:
            r = evaluation_results[col]
            print(f"   {col}: RMSE={r['RMSE']:.4f}, MAE={r['MAE']:.4f}, R2={r['R2']:.4f}")
        
        print(f"\n5. Gerando visualizacao...")
        output_plot = gerar_visualizacao(df_imputed, masked_data, target_cols, evaluation_results, 
                                         f"{estacao_nome} - {modelo_nome}", 
                                         f"{estacao_filename}_{modelo_nome}")
        
        print(f"\n6. Imputacao final nos dados originais...")
        df_final = executar_imputacao(imputer, df_original, target_cols, feature_cols)
        
        print(f"\n7. Exportando resultados...")
        output_files = exportar_resultados(df_final, evaluation_results, f"{estacao_filename}_{modelo_nome}")
        
        modelo_duration = time.time() - modelo_start
        
        print(f"\n8. Registrando no MLflow...")
        
        # Calcular metricas
        avg_rmse = np.mean([m['RMSE'] for m in evaluation_results.values()])
        avg_mae = np.mean([m['MAE'] for m in evaluation_results.values()])
        avg_r2 = np.mean([m['R2'] for m in evaluation_results.values()])
        
        # Registrar no MLflow - cada estacao e uma versao do modelo
        run_name = f"{estacao_nome}_{modelo_nome}_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}"
        
        with mlflow.start_run(run_name=run_name) as run:
            # Tags basicas
            mlflow.set_tags({
                "station_name": estacao_nome,
                "model_type": modelo_nome,
                "experiment_type": "imputation",
                "method": "IterativeImputer",
                "estimator": "RandomForestRegressor"
            })
            
            # Tags com metricas
            mlflow.set_tags({
                f'avg_rmse_{estacao_filename}': str(avg_rmse),
                f'avg_mae_{estacao_filename}': str(avg_mae),
                f'avg_r2_{estacao_filename}': str(avg_r2),
                f'temperatura_rmse_{estacao_filename}': str(evaluation_results['temperatura']['RMSE']),
                f'temperatura_mae_{estacao_filename}': str(evaluation_results['temperatura']['MAE']),
                f'temperatura_r2_{estacao_filename}': str(evaluation_results['temperatura']['R2']),
                f'umidade_rmse_{estacao_filename}': str(evaluation_results['umidade']['RMSE']),
                f'umidade_mae_{estacao_filename}': str(evaluation_results['umidade']['MAE']),
                f'umidade_r2_{estacao_filename}': str(evaluation_results['umidade']['R2']),
                f'velocidade_vento_rmse_{estacao_filename}': str(evaluation_results['velocidade_vento']['RMSE']),
                f'velocidade_vento_mae_{estacao_filename}': str(evaluation_results['velocidade_vento']['MAE']),
                f'velocidade_vento_r2_{estacao_filename}': str(evaluation_results['velocidade_vento']['R2']),
                f'duration_seconds_{estacao_filename}': str(modelo_duration)
            })
            
            # Parametros
            mlflow.log_params({
                'n_estimators': config['n_estimators'],
                'max_depth': config['max_depth'],
                'min_samples_split': 10,
                'min_samples_leaf': 5,
                'max_iter': config['max_iter'],
                'evaluation_ratio': evaluation_ratio,
                'n_records': len(df),
                'n_features': len(feature_cols)
            })
            
            # Metricas numericas para graficos
            mlflow.log_metrics({
                'avg_rmse': avg_rmse,
                'avg_mae': avg_mae,
                'avg_r2': avg_r2,
                'temperatura_rmse': evaluation_results['temperatura']['RMSE'],
                'temperatura_mae': evaluation_results['temperatura']['MAE'],
                'temperatura_r2': evaluation_results['temperatura']['R2'],
                'umidade_rmse': evaluation_results['umidade']['RMSE'],
                'umidade_mae': evaluation_results['umidade']['MAE'],
                'umidade_r2': evaluation_results['umidade']['R2'],
                'velocidade_vento_rmse': evaluation_results['velocidade_vento']['RMSE'],
                'velocidade_vento_mae': evaluation_results['velocidade_vento']['MAE'],
                'velocidade_vento_r2': evaluation_results['velocidade_vento']['R2'],
                'duration_seconds': modelo_duration
            })
            
            # Artifacts
            mlflow.log_artifact(output_plot)
            mlflow.log_artifact(output_files['metrics'])
            
            # Registrar modelo - cada estacao e uma versao
            # Formato: Imputer_Modelo_Leve_PETROLINA (uma versao por execucao)
            model_info = mlflow.sklearn.log_model(
                sk_model=imputer,
                artifact_path="model",
                registered_model_name=f"Imputer_{modelo_nome}_{estacao_filename}"
            )
        
        # Adicionar tags na versao do modelo registrado
        client = mlflow.MlflowClient()
        model_name = f"Imputer_{modelo_nome}_{estacao_filename}"
        
        # Buscar a versao mais recente
        model_versions = client.search_model_versions(f"name='{model_name}'")
        if model_versions:
            latest_version = model_versions[0].version
            
            # Adicionar tags na versao
            client.set_model_version_tag(model_name, latest_version, "station_name", estacao_nome)
            client.set_model_version_tag(model_name, latest_version, "model_type", modelo_nome)
            client.set_model_version_tag(model_name, latest_version, f'avg_rmse_{estacao_filename}', str(avg_rmse))
            client.set_model_version_tag(model_name, latest_version, f'avg_mae_{estacao_filename}', str(avg_mae))
            client.set_model_version_tag(model_name, latest_version, f'avg_r2_{estacao_filename}', str(avg_r2))
            client.set_model_version_tag(model_name, latest_version, f'temperatura_rmse_{estacao_filename}', str(evaluation_results['temperatura']['RMSE']))
            client.set_model_version_tag(model_name, latest_version, f'temperatura_mae_{estacao_filename}', str(evaluation_results['temperatura']['MAE']))
            client.set_model_version_tag(model_name, latest_version, f'temperatura_r2_{estacao_filename}', str(evaluation_results['temperatura']['R2']))
            client.set_model_version_tag(model_name, latest_version, f'umidade_rmse_{estacao_filename}', str(evaluation_results['umidade']['RMSE']))
            client.set_model_version_tag(model_name, latest_version, f'umidade_mae_{estacao_filename}', str(evaluation_results['umidade']['MAE']))
            client.set_model_version_tag(model_name, latest_version, f'umidade_r2_{estacao_filename}', str(evaluation_results['umidade']['R2']))
            client.set_model_version_tag(model_name, latest_version, f'velocidade_vento_rmse_{estacao_filename}', str(evaluation_results['velocidade_vento']['RMSE']))
            client.set_model_version_tag(model_name, latest_version, f'velocidade_vento_mae_{estacao_filename}', str(evaluation_results['velocidade_vento']['MAE']))
            client.set_model_version_tag(model_name, latest_version, f'velocidade_vento_r2_{estacao_filename}', str(evaluation_results['velocidade_vento']['R2']))
            client.set_model_version_tag(model_name, latest_version, f'duration_seconds_{estacao_filename}', str(modelo_duration))
        
        print(f"\n{modelo_nome} processado em {modelo_duration:.2f}s")
        resultados_modelos[modelo_nome] = evaluation_results
    
    duration = time.time() - start_time
    print(f"\nEstacao {estacao_nome} completa (3 modelos) em {duration:.2f}s")
    print("=" * 70)
    
    return resultados_modelos

print("Funcao processar_estacao() definida")

Funcao processar_estacao() definida


## Execucao Automatica para Todas as Estacoes

Loop principal que processa automaticamente todas as estacoes detectadas, registrando metricas individuais e consolidadas no MLflow.

In [14]:
total_start = time.time()
resultados_todas_estacoes = {}

print("\n" + "=" * 70)
print(f"INICIANDO PROCESSAMENTO DE {len(estacoes_disponiveis)} ESTACOES")
print("=" * 70)

for i, estacao_filename in enumerate(estacoes_disponiveis, 1):
    print(f"\n[{i}/{len(estacoes_disponiveis)}] Processando: {estacao_filename.replace('_', ' ')}")
    
    try:
        metricas = processar_estacao(
            estacao_filename=estacao_filename,
            evaluation_ratio=EVALUATION_RATIO,
            random_state=RANDOM_STATE
        )
        resultados_todas_estacoes[estacao_filename] = metricas
        
    except Exception as e:
        print(f"ERRO ao processar {estacao_filename}: {str(e)}")
        continue

total_duration = time.time() - total_start

print("\n" + "=" * 70)
print("RESUMO GERAL DO PROCESSAMENTO")
print("=" * 70)
print(f"\nEstacoes processadas: {len(resultados_todas_estacoes)}/{len(estacoes_disponiveis)}")
print(f"Tempo total: {total_duration:.2f}s ({total_duration/60:.2f} minutos)")

# if resultados_todas_estacoes:
#     print(f"\nMetricas medias por variavel:")
    
#     for var in ['temperatura', 'umidade', 'velocidade_vento']:
#         all_rmse = []
#         all_mae = []
#         all_r2 = []
        
#         for estacao_metricas in resultados_todas_estacoes.values():
#             for modelo_metricas in estacao_metricas.values():
#                 all_rmse.append(modelo_metricas[var]['RMSE'])
#                 all_mae.append(modelo_metricas[var]['MAE'])
#                 all_r2.append(modelo_metricas[var]['R2'])
        
#         print(f"\n  {var.upper()}:")
#         print(f"    RMSE medio: {np.mean(all_rmse):.4f} (std: {np.std(all_rmse):.4f})")
#         print(f"    MAE medio: {np.mean(all_mae):.4f} (std: {np.std(all_mae):.4f})")
#         print(f"    R2 medio: {np.mean(all_r2):.4f} (std: {np.std(all_r2):.4f})")

print("\n" + "=" * 70)
print("PIPELINE COMPLETO CONCLUIDO COM SUCESSO")
print("=" * 70)


INICIANDO PROCESSAMENTO DE 12 ESTACOES

[1/12] Processando: ARCO VERDE

PROCESSANDO ESTACAO: ARCO VERDE

1. Dataset carregado: (43848, 27)
   Registros: 43,848
   Periodo: 2020-01-01 00:00:00 ate 2024-12-31 23:00:00

2. Mascaramento para avaliacao (10.0%):
   temperatura: 2,830 valores mascarados
   umidade: 2,830 valores mascarados
   velocidade_vento: 2,819 valores mascarados

MODELO 1/3: Modelo_Leve

3. Executando imputacao com Modelo_Leve...

4. Calculando metricas de avaliacao:
   temperatura: RMSE=2.2632, MAE=1.7179, R2=0.7219
   umidade: RMSE=9.2121, MAE=6.7499, R2=0.7531
   velocidade_vento: RMSE=1.2909, MAE=1.0092, R2=0.1607

5. Gerando visualizacao...

6. Imputacao final nos dados originais...

7. Exportando resultados...


2025/12/04 12:44:45 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!



8. Registrando no MLflow...


Registered model 'Imputer_Modelo_Leve_ARCO_VERDE' already exists. Creating a new version of this model...
2025/12/04 12:44:47 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Imputer_Modelo_Leve_ARCO_VERDE, version 10
Created version '10' of model 'Imputer_Modelo_Leve_ARCO_VERDE'.



Modelo_Leve processado em 6.59s

MODELO 2/3: Modelo_Medio

3. Executando imputacao com Modelo_Medio...

4. Calculando metricas de avaliacao:
   temperatura: RMSE=2.0076, MAE=1.4530, R2=0.7812
   umidade: RMSE=8.3351, MAE=5.7685, R2=0.7978
   velocidade_vento: RMSE=1.2636, MAE=0.9907, R2=0.1958

5. Gerando visualizacao...

6. Imputacao final nos dados originais...

7. Exportando resultados...


...


Modelo_Medio processado em 13.25s

MODELO 3/3: Modelo_Robusto

3. Executando imputacao com Modelo_Robusto...

4. Calculando metricas de avaliacao:
   temperatura: RMSE=1.3612, MAE=1.0177, R2=0.8661
   umidade: RMSE=6.7523, MAE=4.6943, R2=0.8700
   velocidade_vento: RMSE=1.0683, MAE=0.8377, R2=0.6027

5. Gerando visualizacao...

6. Imputacao final nos dados originais...

7. Exportando resultados...


2025/12/04 12:54:19 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!



8. Registrando no MLflow...


Registered model 'Imputer_Modelo_Robusto_SURUBIM' already exists. Creating a new version of this model...
2025/12/04 12:54:21 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Imputer_Modelo_Robusto_SURUBIM, version 7
Created version '7' of model 'Imputer_Modelo_Robusto_SURUBIM'.



Modelo_Robusto processado em 22.37s

Estacao SURUBIM completa (3 modelos) em 46.58s

RESUMO GERAL DO PROCESSAMENTO

Estacoes processadas: 12/12
Tempo total: 582.39s (9.71 minutos)

PIPELINE COMPLETO CONCLUIDO COM SUCESSO


## Upload de Resultados para S3

Configura o cliente S3 e realiza o upload dos arquivos PKL, CSV, plots e metricas gerados durante o processo de imputacao. Tambem faz upload dos artifacts armazenados pelo MLflow, incluindo modelos treinados e graficos de avaliacao. Os arquivos sao organizados no bucket S3 em estruturas de diretorios para facilitar a recuperacao e analise posterior.

In [15]:
if not S3_BUCKET_NAME or not AWS_ACCESS_KEY_ID or not AWS_SECRET_ACCESS_KEY:
    print("ERRO: Variaveis de ambiente S3 nao configuradas!")
    print("Configure as seguintes variaveis no arquivo .env:")
    print("  - S3_BUCKET_NAME")
    print("  - AWS_ACCESS_KEY_ID")
    print("  - AWS_SECRET_ACCESS_KEY")
    print("  - AWS_REGION (opcional, padrao: us-east-1)")
else:
    s3_client = boto3.client(
        's3',
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
        region_name=AWS_REGION
    )

    def upload_para_s3(local_file, s3_key):
        try:
            s3_client.upload_file(local_file, S3_BUCKET_NAME, s3_key)
            print(f"  Upload concluido: {local_file} -> s3://{S3_BUCKET_NAME}/{s3_key}")
            return True
        except ClientError as e:
            print(f"  Erro no upload de {local_file}: {e}")
            return False

    def upload_artifacts_mlflow_para_s3(experiment_name):
        client = mlflow.MlflowClient()
        experiment = client.get_experiment_by_name(experiment_name)
        
        if not experiment:
            print(f"Experimento '{experiment_name}' nao encontrado")
            return
        
        runs = client.search_runs(experiment_ids=[experiment.experiment_id])
        
        print(f"\nEncontrando artifacts de {len(runs)} runs...")
        
        for run in runs:
            run_id = run.info.run_id
            run_name = run.data.tags.get('mlflow.runName', run_id)
            
            artifacts = client.list_artifacts(run_id)
            
            for artifact in artifacts:
                local_path = f"mlruns/{experiment.experiment_id}/{run_id}/artifacts/{artifact.path}"
                
                s3_key = f"{S3_PREFIX}/mlflow/experiments/{experiment_name}/runs/{run_name}/{artifact.path}"
                
                if os.path.exists(local_path):
                    if os.path.isfile(local_path):
                        upload_para_s3(local_path, s3_key)
                    elif os.path.isdir(local_path):
                        for root, dirs, files in os.walk(local_path):
                            for file in files:
                                file_path = os.path.join(root, file)
                                relative_path = os.path.relpath(file_path, local_path)
                                s3_file_key = f"{s3_key}/{relative_path}"
                                upload_para_s3(file_path, s3_file_key)

    print("Funcoes de upload para S3 definidas")

Funcoes de upload para S3 definidas


In [16]:
if not S3_BUCKET_NAME:
    print("\nAVISO: Upload para S3 pulado - variaveis de ambiente nao configuradas")
    print("Para habilitar o upload, configure o arquivo .env com:")
    print("  S3_BUCKET_NAME=seu-bucket")
    print("  AWS_ACCESS_KEY_ID=sua-access-key")
    print("  AWS_SECRET_ACCESS_KEY=sua-secret-key")
else:
    print("\n" + "=" * 70)
    print("UPLOAD DE RESULTADOS PARA S3")
    print("=" * 70)

    print(f"\n1. Fazendo upload dos arquivos PKL imputados...")
    pkl_files = glob('dados_imputados_*.pkl')
    csv_files = glob('dados_para_update_neon_*.csv')
    plot_files = glob('avaliacao_imputacao_*.png')
    metric_files = glob('metricas_imputacao_*.pkl')

    todos_arquivos = pkl_files + csv_files + plot_files + metric_files

    for arquivo in todos_arquivos:
        s3_key = f"{S3_PREFIX}/resultados/{arquivo}"
        upload_para_s3(arquivo, s3_key)

    print(f"\n  Total de arquivos enviados: {len(todos_arquivos)}")

    print(f"\n2. Fazendo upload dos artifacts do MLflow...")
    upload_artifacts_mlflow_para_s3(EXPERIMENT_NAME)

    print("\n" + "=" * 70)
    print("UPLOAD PARA S3 CONCLUIDO")
    print("=" * 70)


UPLOAD DE RESULTADOS PARA S3

1. Fazendo upload dos arquivos PKL imputados...
  Upload concluido: dados_imputados_OURICURI_Modelo_Leve.pkl -> s3://avd-5b/dados_imputados/resultados/dados_imputados_OURICURI_Modelo_Leve.pkl
  Upload concluido: dados_imputados_SALGUEIRO_Modelo_Robusto.pkl -> s3://avd-5b/dados_imputados/resultados/dados_imputados_SALGUEIRO_Modelo_Robusto.pkl
  Upload concluido: dados_imputados_CARUARU_Modelo_Robusto.pkl -> s3://avd-5b/dados_imputados/resultados/dados_imputados_CARUARU_Modelo_Robusto.pkl
  Upload concluido: dados_imputados_GARANHUNS_Modelo_Medio.pkl -> s3://avd-5b/dados_imputados/resultados/dados_imputados_GARANHUNS_Modelo_Medio.pkl
  Upload concluido: dados_imputados_PETROLINA_Modelo_Medio.pkl -> s3://avd-5b/dados_imputados/resultados/dados_imputados_PETROLINA_Modelo_Medio.pkl
  Upload concluido: dados_imputados_ARCO_VERDE_Modelo_Leve.pkl -> s3://avd-5b/dados_imputados/resultados/dados_imputados_ARCO_VERDE_Modelo_Leve.pkl
  Upload concluido: dados_imputad